# **Библиотеки**

In [ ]:
import librosa
import zipfile
import io
from scipy.io import wavfile
from google.colab import drive
import numpy as np
import pandas as pd

# **Данные**

# Подгрузка данных

In [ ]:
drive.mount('/content/drive')
df=pd.read_csv('drive/MyDrive/Анализ_акустических_событий/train.csv', sep=',',header=None)
dataset = zipfile.ZipFile('drive/MyDrive/Анализ_акустических_событий/audio_train.zip')
labels = df.values.tolist()

Mounted at /content/drive


# Создание словарей с переводом label названий в label цифры и наоборот

In [ ]:
label_to_figure = {}
count = 0
for i in range(len(labels)):
    if i == 0:
        pass
    elif labels[i][1] in label_to_figure:
        pass
    else:
        label_to_figure[labels[i][1]] = count
        count += 1

figure_to_label = {}
for i in label_to_figure.items():
    figure_to_label[i[1]] = i[0]

# Достаём wav файлы из ZIP

**Сразу переводим в Mel**

In [ ]:
def read_from_zip(filename):
    with io.BytesIO() as bytes_buff:
        bytes_data = dataset.read(filename)
        bytes_buff.write(bytes_data)
        bytes_buff.seek(0)
        sr, wave = wavfile.read(bytes_buff)
        mel = librosa.feature.melspectrogram(wave, sr, n_fft=1024, hop_length=1024//4, n_mels=64, fmax=sr // 2)
    return sr, mel

# Создаём Mel признаки

**Параметры GPU**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=5000)

In [ ]:
ZER = np.zeros((len(labels)-1, 120064))

**Получаем признаки**

In [ ]:
import pandas as pd
Y = []
for i in range(len(labels)):
    if not i == 0:
        sr, X_mel = read_from_zip(f'train/{labels[i][0]}')
        for l in range(len(X_mel)):
            ZER[i-1][l*len(X_mel[l]):len(X_mel[l])*(l+1)] = X_mel[l]
        Y.append(label_to_figure[labels[i][1]])
Y = np.array(Y)

In [ ]:
del labels

**Преобразуем признаки**

# **Сохраняем pсl**

In [ ]:
import pickle

output = open('X_train.pkl', 'wb')
pickle.dump(ZER, output)
output.close()

output = open('Y_train.pkl', 'wb')
pickle.dump(Y, output)
output.close()

output = open('X_test.pkl', 'wb')
pickle.dump(X_test, output)
output.close()

output = open('Y_test.pkl', 'wb')
pickle.dump(y_test, output)
output.close()

# **Классификатор**

# Делим данные на траин и валидацию

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from IPython.display import Image
from imutils import paths
import numpy as np
import cv2
import os

In [ ]:
from sklearn.model_selection import train_test_split
ZER, X_test, Y, y_test = train_test_split(ZER, Y, test_size=0.3)

# Обучение

In [ ]:
from sklearn.svm import LinearSVC
model = LinearSVC(random_state = 2000, C = 10)
model.fit(ZER, Y)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=10, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=2000, tol=0.0001,
          verbose=0)

# Тест

In [ ]:
predictions = model.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score
print(f1_score(y_test, predictions, average='macro'))
print(classification_report(y_test, predictions))

0.0964186183092194
              precision    recall  f1-score   support

           0       0.21      0.29      0.24        24
           1       0.08      0.07      0.08        55
           2       0.02      0.05      0.03        20
           3       0.08      0.06      0.07        17
           4       0.22      0.03      0.06        60
           5       0.10      0.12      0.11        51
           6       0.02      0.04      0.02        23
           7       0.17      0.10      0.12        42
           8       0.00      0.00      0.00        45
           9       0.01      0.02      0.01        45
          10       0.25      0.20      0.22        60
          11       0.07      0.06      0.06        47
          12       0.10      0.03      0.04        38
          13       0.81      0.32      0.46        53
          14       0.53      0.55      0.54        38
          15       1.00      0.08      0.14        13
          16       0.20      0.25      0.22        64
        